In [1]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


# Se levanta el split hecho en el NoteBook anterior

In [2]:
X_train = pd.read_csv("X_train")
X_train.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,...,tipodepropiedad_17,tipodepropiedad_18,tipodepropiedad_19,tipodepropiedad_20,tipodepropiedad_21,tipodepropiedad_22,tipodepropiedad_23,tipodepropiedad_24,tipodepropiedad_25,anio_publ
0,0.0,3.0,2.0,3.0,196.0,160.0,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,2014
1,20.0,4.0,2.0,2.0,245.0,196.0,104734.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,2012
2,5.0,3.0,1.0,2.0,108.0,120.0,58952.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,2015
3,10.0,3.0,1.0,3.0,200.0,147.0,49066.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,2016
4,NaN,NaN,0.0,NaN,NaN,250.0,24892.0,19.23339,-99.166991,0.0,...,0,0,0,0,0,0,0,0,0,2016


In [3]:
X_test = pd.read_csv("X_test")

In [4]:
y_train = pd.read_csv("y_train")

In [5]:
y_test = pd.read_csv("y_test")

# Se levantan todos los XGBoost guardados

In [6]:
xgb_created = []

## Atencion! n tiene que coincidir con la cantidad guardada!

In [7]:
n = 100

In [8]:
for i in range(n):
    current_xgb = xgb.XGBRegressor()
    current_xgb.load_model("xgb_{}".format(i))
    xgb_created.append(current_xgb)

# Se arma el DF de precios para entrenar el RF

In [9]:
XGB_ens_train = pd.DataFrame(index = X_train.index.copy())

In [10]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_train)
    XGB_ens_train['pred_{}'.format(i)] = current_price_pred
    print("xgb_{} finished pred".format(i))
XGB_ens_train.head()

xgb_0 finished pred
xgb_1 finished pred
xgb_2 finished pred
xgb_3 finished pred
xgb_4 finished pred
xgb_5 finished pred
xgb_6 finished pred
xgb_7 finished pred
xgb_8 finished pred
xgb_9 finished pred
xgb_10 finished pred
xgb_11 finished pred
xgb_12 finished pred
xgb_13 finished pred
xgb_14 finished pred
xgb_15 finished pred
xgb_16 finished pred
xgb_17 finished pred
xgb_18 finished pred
xgb_19 finished pred
xgb_20 finished pred
xgb_21 finished pred
xgb_22 finished pred
xgb_23 finished pred
xgb_24 finished pred
xgb_25 finished pred
xgb_26 finished pred
xgb_27 finished pred
xgb_28 finished pred
xgb_29 finished pred
xgb_30 finished pred
xgb_31 finished pred
xgb_32 finished pred
xgb_33 finished pred
xgb_34 finished pred
xgb_35 finished pred
xgb_36 finished pred
xgb_37 finished pred
xgb_38 finished pred
xgb_39 finished pred
xgb_40 finished pred
xgb_41 finished pred
xgb_42 finished pred
xgb_43 finished pred
xgb_44 finished pred
xgb_45 finished pred
xgb_46 finished pred
xgb_47 finished pred
xg

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
0,1.960136e+06,2.083753e+06,1983007.875,2016472.000,2.066938e+06,1915978.375,2.050433e+06,1.996088e+06,1964791.625,1999256.375,...,2009666.000,1.930683e+06,1998047.000,2035771.375,2074387.250,2.030952e+06,2093608.125,1951871.125,1.933595e+06,2.025616e+06
1,1.730548e+06,1.807416e+06,1977822.500,1908898.375,1.893285e+06,1680147.375,1.738651e+06,1.737802e+06,1695895.625,1792483.000,...,1797786.750,1.844899e+06,1808157.500,2035555.625,2023181.000,1.711074e+06,1682701.625,1685264.750,1.746887e+06,1.817009e+06
2,9.264328e+05,1.002248e+06,1048980.000,962219.500,9.933976e+05,967627.625,9.675136e+05,9.792166e+05,960865.000,966808.250,...,956611.625,9.780559e+05,1003635.125,959082.500,1030883.375,9.201554e+05,957448.750,975332.875,9.624619e+05,1.005428e+06
3,2.129169e+06,2.091265e+06,2077858.750,2121069.000,2.091269e+06,2045930.500,2.055335e+06,1.923376e+06,1980416.125,2285495.750,...,2136472.500,1.974236e+06,2117227.500,2062277.625,2098303.000,2.135436e+06,2021440.000,2185763.000,2.185930e+06,2.142510e+06
4,2.380556e+06,2.442019e+06,2605916.250,2426843.250,2.968246e+06,2784467.750,2.320495e+06,2.604902e+06,2541640.500,2832155.750,...,3120014.500,1.606927e+06,2407676.000,2641577.250,2767342.250,2.190036e+06,2138027.750,2244863.750,2.627987e+06,2.607003e+06


In [11]:
len(XGB_ens_train)

168000

In [12]:
len(y_train)

168000

## Exporto el XGB_ens_train para poder buscar hiperparametros

In [13]:
XGB_ens_train.to_csv("XGB_ens_train", index = False)

# Entreno el XGB con XGB_ens_train

In [14]:
model_XGB = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1.0, gamma=1,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=40, min_child_weight=20, missing=None, n_estimators=200,
       n_jobs=-1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1.0, verbosity=1)
model_XGB.fit(XGB_ens_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1.0, gamma=1,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=40, min_child_weight=20, missing=None, n_estimators=200,
       n_jobs=-1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1.0, verbosity=1)

# Testeo el modelo final

## Armo el DF para X_test

In [15]:
XGB_ens_test = pd.DataFrame(index = X_test.index.copy())


In [16]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_test)
    XGB_ens_test['pred_{}'.format(i)] = current_price_pred
XGB_ens_test.head()

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
0,1615089.375,1.559345e+06,1.504908e+06,1.674037e+06,1687985.625,1.509940e+06,1552491.50,1683150.250,1.572348e+06,1.801518e+06,...,1.613074e+06,1553691.50,1.762813e+06,1.628762e+06,1481052.750,1.838269e+06,1.517993e+06,1743967.375,1933610.875,1.536322e+06
1,2583035.750,2.401464e+06,2.523978e+06,2.278840e+06,2419261.000,2.580714e+06,2474698.00,2321236.750,2.884982e+06,2.341641e+06,...,2.421718e+06,2440893.75,2.444819e+06,2.364272e+06,2251434.750,2.744593e+06,2.609640e+06,2578859.750,2431762.250,2.372205e+06
2,2881287.750,2.994866e+06,2.457351e+06,3.109494e+06,3105666.500,2.958059e+06,2739394.50,2705363.250,2.763808e+06,2.667100e+06,...,2.866243e+06,2672559.00,2.574218e+06,2.804133e+06,2394266.500,2.435635e+06,2.699949e+06,2737217.500,2918107.250,2.853083e+06
3,1429415.125,1.321188e+06,1.515572e+06,1.244933e+06,1435273.875,1.416682e+06,1398179.25,1378882.625,1.208501e+06,1.560161e+06,...,1.450961e+06,1432722.75,1.349617e+06,1.268668e+06,1400775.375,1.488959e+06,1.441191e+06,1385087.875,1603957.625,1.517252e+06
4,365705.625,3.286927e+05,2.602707e+05,3.916093e+05,285437.000,1.959263e+05,309702.50,359414.125,2.726801e+05,3.630856e+05,...,3.353237e+05,289292.00,3.420312e+05,2.942465e+05,277769.875,2.264651e+05,4.286672e+05,322354.750,301506.125,2.237315e+05


## Se hacen las dos predicciones necesarias

In [17]:
pred_train = model_XGB.predict(XGB_ens_train)
pred_test = model_XGB.predict(XGB_ens_test)

### Metrica de Mean Absolute Error (La de Kaggle) ?? (Hay que preguntar)

In [18]:
mae_train = np.sqrt(sklearn.metrics.mean_absolute_error(y_train, pred_train))
mae_test = np.sqrt(sklearn.metrics.mean_absolute_error(y_test, pred_test))
#print(f"MAE train: {mea_train:.5f}")
#print(f"MAE test: {mea_test:.5f}")
print("MAE train: "+str(mae_train))
print("MAE test: "+str(mae_test))

MAE train: 413.9856008194318
MAE test: 783.9149079358771


#### Overfittea una bocha! Pero eso quiere decir que hay mucho rango para mejorar :)

In [19]:
model_XGB.feature_importances_

array([0.00170509, 0.00180482, 0.00163001, 0.00173704, 0.00159478,
       0.00146388, 0.00310106, 0.00153546, 0.0018615 , 0.00163194,
       0.00147223, 0.008101  , 0.00155398, 0.00167169, 0.00144396,
       0.00496454, 0.00148135, 0.00219048, 0.00175162, 0.00161225,
       0.00594892, 0.00153099, 0.00150551, 0.01208959, 0.00183358,
       0.00644055, 0.05864176, 0.0036491 , 0.00157442, 0.00406266,
       0.4228068 , 0.00450117, 0.0027635 , 0.01051538, 0.00157009,
       0.00855451, 0.00171301, 0.00215963, 0.00161869, 0.00204664,
       0.00162697, 0.00174817, 0.00175738, 0.00148731, 0.00161249,
       0.00156684, 0.00509131, 0.00207694, 0.00175436, 0.0016738 ,
       0.20016518, 0.00178368, 0.00181365, 0.00186609, 0.00255212,
       0.02932397, 0.00187058, 0.00165521, 0.02213236, 0.00210592,
       0.00164226, 0.00261135, 0.00236743, 0.00217962, 0.00137476,
       0.00203196, 0.00175843, 0.00172234, 0.00160058, 0.00165791,
       0.00969865, 0.00159288, 0.00152894, 0.00440354, 0.00770

# Se genera el archivo para subir a Kaggle

In [20]:
test_set = pd.read_csv("test_set_xgb.csv")

In [21]:
test_set.columns

Index(['id', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio',
       'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'tipodepropiedad_1', 'tipodepropiedad_2',
       'tipodepropiedad_3', 'tipodepropiedad_4', 'tipodepropiedad_5',
       'tipodepropiedad_6', 'tipodepropiedad_7', 'tipodepropiedad_8',
       'tipodepropiedad_9', 'tipodepropiedad_10', 'tipodepropiedad_11',
       'tipodepropiedad_12', 'tipodepropiedad_13', 'tipodepropiedad_14',
       'tipodepropiedad_15', 'tipodepropiedad_16', 'tipodepropiedad_17',
       'tipodepropiedad_18', 'tipodepropiedad_19', 'tipodepropiedad_20',
       'tipodepropiedad_21', 'tipodepropiedad_22', 'tipodepropiedad_23',
       'tipodepropiedad_24', 'tipodepropiedad_25', 'anio_publ'],
      dtype='object')

In [22]:
X_test_set = test_set.loc[:, test_set.columns != 'id']

## Armo el DF con las predicciones de test

In [23]:
XGB_ens_test_set = pd.DataFrame(index = test_set.index.copy())

In [24]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_test_set)
    XGB_ens_test_set['pred_{}'.format(i)] = current_price_pred
    print("xgb_{} finished pred".format(i))
XGB_ens_test_set.head()

xgb_0 finished pred
xgb_1 finished pred
xgb_2 finished pred
xgb_3 finished pred
xgb_4 finished pred
xgb_5 finished pred
xgb_6 finished pred
xgb_7 finished pred
xgb_8 finished pred
xgb_9 finished pred
xgb_10 finished pred
xgb_11 finished pred
xgb_12 finished pred
xgb_13 finished pred
xgb_14 finished pred
xgb_15 finished pred
xgb_16 finished pred
xgb_17 finished pred
xgb_18 finished pred
xgb_19 finished pred
xgb_20 finished pred
xgb_21 finished pred
xgb_22 finished pred
xgb_23 finished pred
xgb_24 finished pred
xgb_25 finished pred
xgb_26 finished pred
xgb_27 finished pred
xgb_28 finished pred
xgb_29 finished pred
xgb_30 finished pred
xgb_31 finished pred
xgb_32 finished pred
xgb_33 finished pred
xgb_34 finished pred
xgb_35 finished pred
xgb_36 finished pred
xgb_37 finished pred
xgb_38 finished pred
xgb_39 finished pred
xgb_40 finished pred
xgb_41 finished pred
xgb_42 finished pred
xgb_43 finished pred
xgb_44 finished pred
xgb_45 finished pred
xgb_46 finished pred
xgb_47 finished pred
xg

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
0,5.067982e+06,4.819608e+06,5.088392e+06,4.934705e+06,4.648890e+06,5.054410e+06,5026901.50,5.363598e+06,4.745803e+06,5.017534e+06,...,5.541759e+06,5118448.500,5.265462e+06,5.403524e+06,4.984222e+06,5353598.500,5.214220e+06,4898149.000,5.097256e+06,5.651972e+06
1,8.694842e+05,9.149268e+05,7.161017e+05,8.025454e+05,6.948494e+05,8.085341e+05,930262.75,7.303592e+05,8.801612e+05,7.972906e+05,...,6.155112e+05,811765.875,8.143768e+05,7.447892e+05,8.100305e+05,717957.375,7.808083e+05,849581.625,8.987915e+05,8.818965e+05
2,2.184057e+06,2.600893e+06,2.457338e+06,2.462860e+06,2.393930e+06,2.421340e+06,2402905.50,2.371461e+06,2.331526e+06,2.454802e+06,...,2.511733e+06,2387544.250,2.408880e+06,2.279167e+06,2.450404e+06,2239084.500,2.510001e+06,2076690.000,2.419792e+06,2.429706e+06
3,1.456018e+06,1.655609e+06,1.504390e+06,1.521739e+06,1.518617e+06,1.757235e+06,1577588.75,1.361244e+06,1.379325e+06,1.530791e+06,...,1.421200e+06,1380316.250,1.477788e+06,1.601011e+06,1.544672e+06,1241860.500,1.549970e+06,1592738.250,1.640370e+06,1.558910e+06
4,6.096271e+05,4.975887e+05,5.342101e+05,6.061721e+05,5.486543e+05,5.531181e+05,534087.25,5.273641e+05,5.288957e+05,5.555491e+05,...,5.662944e+05,510410.375,5.765152e+05,5.609772e+05,5.245491e+05,569735.625,5.779522e+05,583801.875,5.079645e+05,5.439124e+05


In [25]:
pred_a_kaggle = model_XGB.predict(XGB_ens_test_set)

In [26]:
df_a_kaggle = pd.DataFrame()
df_a_kaggle['id'] = test_set['id']
df_a_kaggle['target'] = pred_a_kaggle
df_a_kaggle.head()

,id,target
0,4941,5.854051e+06
1,51775,8.423851e+05
2,115253,2.211423e+06
3,299321,1.334854e+06
4,173570,6.062748e+05


In [27]:
df_a_kaggle.to_csv("subir_a_kaggle", header = True, index = False)